In [1]:
import pandas as pd
import logging

# Log fayl manzili
log_path = r"C:\Users\Rasulbek907\Desktop\Final_Project\Log\feature_selection.log"

# Log sozlamalari
logging.basicConfig(
    filename=log_path,
    filemode='a',  # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

try:
    logging.info("CSV fayl o'qilmoqda:...")
    df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Final_Project\Data\Preprosessed_data\Clustering.csv")
    logging.info(f"Fayl muvaffaqiyatli o'qildi. Satırlar soni: {len(df)} ustunlar soni: {len(df.columns)}")
except Exception as e:
    logging.error(f"CSV faylni o'qishda xatolik: {e}")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2022 entries, 0 to 2021
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0.1        2022 non-null   int64  
 1   Unnamed: 0          2022 non-null   int64  
 2   name                2022 non-null   object 
 3   birth_date          2022 non-null   object 
 4   birth_place         2022 non-null   object 
 5   death_date          2022 non-null   object 
 6   death_place         2022 non-null   object 
 7   occupation          2022 non-null   object 
 8   awards              2022 non-null   object 
 9   alma_mater          2022 non-null   object 
 10  education           2022 non-null   object 
 11  spouse              2022 non-null   object 
 12  children            2022 non-null   object 
 13  occupation_cluster  2022 non-null   int64  
 14  birth_year          2022 non-null   float64
 15  death_year          2022 non-null   float64
 16  life_s

In [3]:
df = df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)

In [4]:
df['life_span'] = df['life_span'].abs()

In [11]:
df.head()

,name,birth_date,birth_place,death_date,death_place,occupation,awards,alma_mater,education,spouse,children,occupation_cluster,birth_year,death_year,life_span,life_span_cluster,edu_award_cluster,bio_cluster
0,Isaac Newton,{{Birth date|df=y|1643|01|04}},"{{nowrap|[[Woolsthorpe-by-Colsterworth]],}} Li...",{{Death date and age|df=y|1727|03|31|1643|01|04}},"[[Kensington]], Middlesex, England",Composer,{{unbulleted list | [[Fellow of the Royal Soci...,[[King's College London]],"[[Trinity College, Cambridge]] ([[Bachelor of ...",{{plainlist|,2,0,1643.0,1727.0,84.0,2.0,3,1
1,Marie Curie,{{birth date|df=y|1867|11|7}},"[[Warsaw]], [[Congress Poland|Poland]], Russia...",{{death date and age|df=y|1934|7|4|1867|11|7}},"[[Passy, Haute-Savoie|Passy]], [[Rhône-Alpes]]...",Composer,}},[[University of Paris]],[[Visva-Bharati University]] (dropped out)<ref...,{{plainlist|,2,0,1867.0,1934.0,67.0,2.0,1,0
2,Albert Einstein,{{Birth date|df=yes|1879|3|14}},"[[Ulm]], [[Kingdom of Württemberg]], Germany<r...",{{Death date and age|df=yes|1955|4|18|1879|3|14}},"[[Princeton, New Jersey]], U.S.",Composer,}},[[King's College London]],[[Visva-Bharati University]] (dropped out)<ref...,{{plainlist|,{{flatlist|,0,1879.0,1955.0,76.0,2.0,1,0
3,The Countess of Lovelace,{{birth date|df=y|1815|12|10}},"London, England",{{death date and age|df=y|1852|11|27|1815|12|10}},"[[Marylebone]], London, England",Composer,}},[[King's College London]],[[Visva-Bharati University]] (dropped out)<ref...,"{{marriage|[[William King-Noel, 1st Earl of Lo...",{{plainlist|,0,1815.0,1852.0,37.0,2.0,1,0
4,Leonardo da Vinci,{{birth date|df=yes|1452|04|15}},"possibly [[Anchiano]],{{efn|name=Birthplace}} ...",{{death date and age|df=yes|1519|5|2|1452|4|15}},"[[Clos Lucé]], [[Amboise]], {{awrap|[[Kingdom ...",Composer,}},[[King's College London]],Studio of [[Andrea del Verrocchio]],{{plainlist|,2,0,1452.0,1519.0,67.0,5.0,3,3


In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df_fe = df.copy()

# datetime formatga o'tkazish (agar kerak bo'lsa)
df_fe['birth_date'] = pd.to_datetime(df_fe['birth_date'], errors='coerce')
df_fe['death_date'] = pd.to_datetime(df_fe['death_date'], errors='coerce')

# Yangi ustun: umr davomiyligi (agar allaqachon bo'lmasa)
# df_fe['life_span'] = (df_fe['death_date'] - df_fe['birth_date']).dt.days / 365.25

# Kategorik ustunlarni label encoding qilish (modelga kirish uchun)
categorical_cols = ['name', 'birth_place', 'death_place', 'occupation', 'awards',
                    'alma_mater', 'education', 'spouse', 'children']

for col in categorical_cols:
    le = LabelEncoder()
    df_fe[col] = le.fit_transform(df_fe[col].astype(str))

# Maqsad ustuni
target = df_fe['life_span']

# Tavsiflovchi ustunlar (target ustunidan tashqari hamma ustunlar)
features = df_fe.drop(columns=['life_span', 'birth_date', 'death_date'])

print(features.head())
print(target.head())


   name  birth_place  death_place  occupation  awards  alma_mater  education  \
0   334          616          202          10     122          29         85   
1   474          575          261          10     135          83        107   
2    31          556          270          10     135          29        107   
3   688           11          228          10     135          29        107   
4   440          602          135          10     135          29          5   

   spouse  children  occupation_cluster  birth_year  death_year  \
0     233        12                   0      1643.0      1727.0   
1     233        12                   0      1867.0      1934.0   
2     233       115                   0      1879.0      1955.0   
3     227       130                   0      1815.0      1852.0   
4     233        12                   0      1452.0      1519.0   

   life_span_cluster  edu_award_cluster  bio_cluster  
0                2.0                  3            1  
1     

C:\Users\Rasulbek907\AppData\Local\Temp\ipykernel_14668\2650285840.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_fe['birth_date'] = pd.to_datetime(df_fe['birth_date'], errors='coerce')
C:\Users\Rasulbek907\AppData\Local\Temp\ipykernel_14668\2650285840.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_fe['death_date'] = pd.to_datetime(df_fe['death_date'], errors='coerce')


In [9]:
df_fe.head()

,name,birth_date,birth_place,death_date,death_place,occupation,awards,alma_mater,education,spouse,children,occupation_cluster,birth_year,death_year,life_span,life_span_cluster,edu_award_cluster,bio_cluster
0,334,NaT,616,NaT,202,10,122,29,85,233,12,0,1643.0,1727.0,84.0,2.0,3,1
1,474,NaT,575,NaT,261,10,135,83,107,233,12,0,1867.0,1934.0,67.0,2.0,1,0
2,31,NaT,556,NaT,270,10,135,29,107,233,115,0,1879.0,1955.0,76.0,2.0,1,0
3,688,NaT,11,NaT,228,10,135,29,107,227,130,0,1815.0,1852.0,37.0,2.0,1,0
4,440,NaT,602,NaT,135,10,135,29,5,233,12,0,1452.0,1519.0,67.0,5.0,3,3


In [10]:
df_fe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2022 entries, 0 to 2021
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   name                2022 non-null   int64         
 1   birth_date          28 non-null     datetime64[ns]
 2   birth_place         2022 non-null   int64         
 3   death_date          0 non-null      datetime64[ns]
 4   death_place         2022 non-null   int64         
 5   occupation          2022 non-null   int64         
 6   awards              2022 non-null   int64         
 7   alma_mater          2022 non-null   int64         
 8   education           2022 non-null   int64         
 9   spouse              2022 non-null   int64         
 10  children            2022 non-null   int64         
 11  occupation_cluster  2022 non-null   int64         
 12  birth_year          2022 non-null   float64       
 13  death_year          2022 non-null   float64     